In [78]:
from keras.engine import Input
from keras.engine import Model
from keras.layers import Dense, TimeDistributed, LSTM, Dropout, Activation, \
    Lambda, concatenate, add
import numpy as np

data = np.load('drum.npy')
data = data[:30]

In [79]:
X = np.transpose(data, (0, 2, 1))
X.shape

(30, 64, 3)

In [62]:
def split_features(X):
    N = X.shape[0]
    timesteps = 8
    X_left = np.zeros((N * 144, timesteps, 3))
    X_right = np.zeros((N * 144, timesteps, 3))
    X_center = np.zeros((N * 144, 2))
    y = np.zeros((N * 144, 2))
    
    for k in range(N):
        for i in range(8, 56):
            for j in range(3):
                # print(k, i ,j, k*144+i*3+j)
                X_left[k*144+(i-8)*3+j] = X[k, i-8:i, :]
                X_right[k*144+(i-8)*3+j] = X[k, i+1:i+9, :]
            
                if j == 0:
                    X_center[k*144+(i-8)*3+j] = X[k, i, 1:3]
                elif j == 1:
                    X_center[k*144+(i-8)*3+j] = X[k, i, 0::2]
                else:
                    X_center[k*144+(i-8)*3+j] = X[k, i, :2]
            
                y[k*144+(i-8)*3+j, int(X[k, i, j])] = 1 
    
    return X_left, X_center, X_right, y

X_left, X_center, X_right, y = split_features(X)

In [5]:
timesteps = 8

left_features = Input(shape=(timesteps, 3),
                          name='left_features')
right_features = Input(shape=(timesteps, 3),
                           name='right_features')
central_features = Input(shape=(2,), name='central_features')

In [20]:
num_dense = 20

embedding_left = Dense(input_dim=3, output_dim=num_dense, name='embedding_left')
embedding_right = Dense(input_dim=3, output_dim=num_dense, name='embedding_right')
predictions_left = TimeDistributed(embedding_left)(left_features)
predictions_right = TimeDistributed(embedding_right)(right_features)

predictions_center = Dense(num_dense, activation='relu')(central_features)
predictions_center = Dense(num_dense, activation='relu')(predictions_center)

predictions_left.shape

/Users/honglinma/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:3: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(input_dim=3, units=20, name="embedding_left")`
  app.launch_new_instance()
/Users/honglinma/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:4: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(input_dim=3, units=20, name="embedding_right")`


TensorShape([Dimension(None), Dimension(8), Dimension(20)])

In [21]:
# deep LSTM:
predictions_left = LSTM(32, return_sequences=True, name='lstm_left1')(predictions_left)
predictions_right = LSTM(32, return_sequences=True, name='lstm_right1')(predictions_right)
predictions_left = LSTM(32, name='lstm_left2')(predictions_left)
predictions_right = LSTM(32,name='lstm_right2')(predictions_right)

predictions = concatenate([predictions_left, predictions_center, predictions_right])
predictions = Dense(num_dense, activation='relu')(predictions)
prediction = Dense(2, activation='softmax', name='prediction')(predictions)

In [22]:
prediction.shape

TensorShape([Dimension(None), Dimension(2)])

In [23]:
model = Model(input=[left_features, central_features, right_features], output=prediction)
model.compile(optimizer='adam', loss={'prediction': 'categorical_crossentropy'})
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
left_features (InputLayer)      (None, 8, 3)         0                                            
__________________________________________________________________________________________________
right_features (InputLayer)     (None, 8, 3)         0                                            
__________________________________________________________________________________________________
time_distributed_8 (TimeDistrib (None, 8, 20)        80          left_features[0][0]              
__________________________________________________________________________________________________
central_features (InputLayer)   (None, 2)            0                                            
__________________________________________________________________________________________________
time_distr

/Users/honglinma/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:1: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=[<tf.Tenso..., outputs=Tensor("pr...)`
  if __name__ == '__main__':


In [47]:
num_epoch = 100
batch_size = 64
model.fit([X_left, X_center, X_right], y,
          batch_size=batch_size,
          epochs= num_epoch)

Epoch 1/100
4320/4320 [==============================] - 1s 264us/step - loss: 0.0596
Epoch 2/100
4320/4320 [==============================] - 1s 272us/step - loss: 0.0534
Epoch 3/100
4320/4320 [==============================] - 1s 274us/step - loss: 0.0533
Epoch 4/100
4320/4320 [==============================] - 1s 274us/step - loss: 0.0553
Epoch 5/100
4320/4320 [==============================] - 1s 277us/step - loss: 0.0529
Epoch 6/100
4320/4320 [==============================] - 1s 279us/step - loss: 0.0527
Epoch 7/100
4320/4320 [==============================] - 1s 282us/step - loss: 0.0523
Epoch 8/100
4320/4320 [==============================] - 1s 277us/step - loss: 0.0531
Epoch 9/100
4320/4320 [==============================] - 1s 281us/step - loss: 0.0527
Epoch 10/100
4320/4320 [==============================] - 1s 294us/step - loss: 0.0517
Epoch 11/100
4320/4320 [==============================] - 1s 287us/step - loss: 0.0526
Epoch 12/100
4320/4320 [============================

4320/4320 [==============================] - 1s 331us/step - loss: 0.0524
Epoch 96/100
4320/4320 [==============================] - 1s 333us/step - loss: 0.0600
Epoch 97/100
4320/4320 [==============================] - 1s 325us/step - loss: 0.0522
Epoch 98/100
4320/4320 [==============================] - 1s 331us/step - loss: 0.0539
Epoch 99/100
4320/4320 [==============================] - 1s 327us/step - loss: 0.0497
Epoch 100/100
4320/4320 [==============================] - 1s 320us/step - loss: 0.0498


In [86]:
idx = np.random.choice(30, 5)
X_gen = X[idx]
X_gen[:, 8:56, :] = 0


In [65]:
a = np.zeros((3, 64, 3))
a_l, a_c, a_r, _ = split_features(a)
p = model.predict([a_l, a_c, a_r])
p.shape

432/432 [==============================] - 0s 100us/step


(432, 2)

In [90]:
M = 1000

for m in range(M):
    print("Step:", m)
    for k in range(5):
        X_gen_left, X_gen_center, X_gen_right, _ = split_features(X_gen)
        i = np.random.choice(48, 1)
        j = np.random.choice(3, 1)
        sample_scores = model.predict([X_gen_left, X_gen_center, X_gen_right], verbose=0)[k*144+i*3+j]
        sample_scores = np.asarray(sample_scores).astype('float64')
        exp_sample_scores = np.exp(np.log(sample_scores))            
        preds = (sample_scores / np.sum(exp_sample_scores))[0]
        result = np.argmax(np.random.multinomial(20, preds, 1))
        X_gen[k, i+8, j] = result
        

Step: 0
Step: 1
Step: 2
Step: 3
Step: 4
Step: 5
Step: 6
Step: 7
Step: 8
Step: 9
Step: 10
Step: 11
Step: 12
Step: 13
Step: 14
Step: 15
Step: 16
Step: 17
Step: 18
Step: 19
Step: 20
Step: 21
Step: 22
Step: 23
Step: 24
Step: 25
Step: 26
Step: 27
Step: 28
Step: 29
Step: 30
Step: 31
Step: 32
Step: 33
Step: 34
Step: 35
Step: 36
Step: 37
Step: 38
Step: 39
Step: 40
Step: 41
Step: 42
Step: 43
Step: 44
Step: 45
Step: 46
Step: 47
Step: 48
Step: 49
Step: 50
Step: 51
Step: 52
Step: 53
Step: 54
Step: 55
Step: 56
Step: 57
Step: 58
Step: 59
Step: 60
Step: 61
Step: 62
Step: 63
Step: 64
Step: 65
Step: 66
Step: 67
Step: 68
Step: 69
Step: 70
Step: 71
Step: 72
Step: 73
Step: 74
Step: 75
Step: 76
Step: 77
Step: 78
Step: 79
Step: 80
Step: 81
Step: 82
Step: 83
Step: 84
Step: 85
Step: 86
Step: 87
Step: 88
Step: 89
Step: 90
Step: 91
Step: 92
Step: 93
Step: 94
Step: 95
Step: 96
Step: 97
Step: 98
Step: 99
Step: 100
Step: 101
Step: 102
Step: 103
Step: 104
Step: 105
Step: 106
Step: 107
Step: 108
Step: 109
Step: 110


Step: 831
Step: 832
Step: 833
Step: 834
Step: 835
Step: 836
Step: 837
Step: 838
Step: 839
Step: 840
Step: 841
Step: 842
Step: 843
Step: 844
Step: 845
Step: 846
Step: 847
Step: 848
Step: 849
Step: 850
Step: 851
Step: 852
Step: 853
Step: 854
Step: 855
Step: 856
Step: 857
Step: 858
Step: 859
Step: 860
Step: 861
Step: 862
Step: 863
Step: 864
Step: 865
Step: 866
Step: 867
Step: 868
Step: 869
Step: 870
Step: 871
Step: 872
Step: 873
Step: 874
Step: 875
Step: 876
Step: 877
Step: 878
Step: 879
Step: 880
Step: 881
Step: 882
Step: 883
Step: 884
Step: 885
Step: 886
Step: 887
Step: 888
Step: 889
Step: 890
Step: 891
Step: 892
Step: 893
Step: 894
Step: 895
Step: 896
Step: 897
Step: 898
Step: 899
Step: 900
Step: 901
Step: 902
Step: 903
Step: 904
Step: 905
Step: 906
Step: 907
Step: 908
Step: 909
Step: 910
Step: 911
Step: 912
Step: 913
Step: 914
Step: 915
Step: 916
Step: 917
Step: 918
Step: 919
Step: 920
Step: 921
Step: 922
Step: 923
Step: 924
Step: 925
Step: 926
Step: 927
Step: 928
Step: 929
Step: 930


In [93]:
X_gen.shape
X_gen = np.transpose(X_gen, (0, 2, 1))
X_gen

array([[[ 1.,  0.,  1.,  0.,  1.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,
          0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
          0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
          0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
          0.,  0.,  0.,  0.,  1.,  0.,  1.,  0.,  1.,  0.,  1.,  0.],
        [ 0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
          0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
          0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
          0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
          0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  1.,  0.,  0.,  0.],
        [ 1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
          0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
          0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
          0.,  0.,  0.,  0.,  0.,  0.,  0.,  0